# Read and clean a single file

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'yuma_bucket'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [2]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [3]:
raw_data_dic = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'datos_abiertos_cons_diciembre_2019.csv')
print(raw_data_dic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16243 entries, 0 to 16242
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     16243 non-null  object
 1   FECHA_DESPACHO_518   16243 non-null  object
 2   CODIGO DE LOCALIDAD  16243 non-null  int64 
 3   LOCALIDAD            16243 non-null  object
 4   EDAD                 16243 non-null  int64 
 5   UNIDAD               16243 non-null  object
 6   GENERO               16243 non-null  object
 7   RED                  16243 non-null  object
 8   TIPO_INCIDENTE       16243 non-null  object
 9   PRIORIDAD            16243 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [4]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data_dic.rename(
    columns = {
        'NUMERO_INCIDENTE'                 : 'FECHA_INCIDENTE',
        'FECHA_DESPACHO_518'               : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
    },
    inplace=True
)
raw_data_dic.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


### Check values per field
--------
1. FECHA_INICIO_DESPLAZAMIENTO-MOVIL
2. LOCALIDAD
3. UNIDAD
4. GENERO
5. RED
6. TIPO_INCIDENTE
7. PRIORIDAD

In [5]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
raw_data_dic[col] = pd.to_datetime(raw_data_dic[col], format='%Y/%m/%d %H:%M') 
raw_data_dic[col].value_counts
raw_data_dic.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [6]:
col = 'LOCALIDAD'
raw_data_dic[col] = raw_data_dic[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False) 

Kennedy               2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [8]:
col = 'UNIDAD'
raw_data_dic[col] = raw_data_dic[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False) 

SIN/DATO    8286
Años        7889
Meses         45
Horas         13
Dias          10
Name: UNIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [9]:
col = 'GENERO'
raw_data_dic[col] = raw_data_dic[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False) 

SIN/DATO     8286
MASCULINO    4405
FEMENINO     3552
Name: GENERO, dtype: int64

In [10]:
col = 'RED'
raw_data_dic[col] = raw_data_dic[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False) 

Norte      7114
Sur        6282
Privada    2847
Name: RED, dtype: int64

In [11]:
col = 'TIPO_INCIDENTE'
raw_data_dic[col] = raw_data_dic[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False)

Heridos accidentales                      7390
Inconsciente / paro cardiorespiratorio    1743
Evento respiratorio                       1404
Convulsión                                1027
Enfermo                                    873
Trastorno mental                           650
Dolor torácico                             611
Intento de Suicidio                        543
Caída de Altura                            377
Patología ginecobstétrica                  371
Accidente Cerebro\nVascular                365
Síntomas gastrointestinales                275
Intoxicación                               153
S/D                                        138
Accidente Tránsito con Heridos/Muertos     108
Amenaza de\nSuicidio                        95
Violencia Sexual                            28
Sangrado Vaginal                            24
Lesiones Personales                         23
Quemaduras                                  16
Electrocución / Rescate                     10
Accidente Cer

In [15]:
col = 'PRIORIDAD'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data_dic[col].value_counts(dropna=False)

2        10668
1         3128
3         1922
ALTA       300
4          197
MEDIA       23
BAJA         5
Name: PRIORIDAD, dtype: int64

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [23]:
project_dir

'/home/jupyter/ESEIT_BigData'

In [16]:
profile = ProfileReport(df=raw_data_dic)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_diciembre_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [17]:
print('Number of rows before cleaning:', raw_data_dic.shape[0] )
raw_data_dic.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data_dic.shape[0] )

Number of rows before cleaning: 16243
Number of rows after cleaning: 16162


### Save the final table
use to_csv to save the table

In [19]:
# saving in a local directory
raw_data_dic.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_diciembre_2019.csv', encoding='latin1', sep=';', index=False)